In [6]:
# Dependencies
import pandas as pd
from splinter import Browser
from bs4 import BeautifulSoup as bs
import pymongo
import requests
from webdriver_manager.chrome import ChromeDriverManager

In [2]:
# Set up of Chrome webdriver
executable_path = {"executable_path": ChromeDriverManager().install()}
browser = Browser('chrome', **executable_path, headless=False)

[WDM] - Current google-chrome version is 86.0.4240
[WDM] - Get LATEST driver version for 86.0.4240


[WDM] - Driver [/Users/brykobe24/.wdm/drivers/chromedriver/mac64/86.0.4240.22/chromedriver] found in cache


In [3]:
# Scraping preparation and store data in dictionary
get_mars_data = {}
url = 'https://mars.nasa.gov/news/'
response= requests.get(url)
soup = bs(response.text, 'html.parser')

# NASA Mars News:
Scrape the NASA Mars News Site and collect the latest News Title and Paragraph Text. Assign the text to variables that you can reference later.

In [4]:
news_title = soup.find('div', class_= 'content_title').text
news_paragraph = soup.find('div', class_= 'rollover_description_inner').text 
print('Most Recent Nasa News Article...')
print(f'Title: {news_title}')
print(f'Substance: {news_paragraph}')

# Push values to mars dictionary
get_mars_data['recent_news'] = news_title
get_mars_data['recent_news_substance'] = news_paragraph

Most Recent Nasa News Article...
Title: 

NASA to Broadcast Mars 2020 Perseverance Launch, Prelaunch Activities


Substance: 
Starting July 27, news activities will cover everything from mission engineering and science to returning samples from Mars to, of course, the launch itself.



# JPL Mars Space Images - Featured Image
Visit the url for JPL Featured Space Image here. Use splinter to navigate the site and find the image url for the current Featured Mars Image and assign the url string to a variable called featured_image_url.

In [9]:
# Url we will be scraping images from
url = 'https://www.jpl.nasa.gov/spaceimages/?search=&category=Mars'
base_url = 'https://www.jpl.nasa.gov'

response = requests.get(url)
soup = bs(response.text, 'html.parser')

splint_url = base_url + soup.find('a', class_="button fancybox")["data-fancybox-href"]

print(f"URL to Featured Nasa Image: {splint_url}")

URL to Featured Nasa Image: https://www.jpl.nasa.gov/spaceimages/images/mediumsize/PIA19631_ip.jpg


# Mars Facts
Visit the Mars Facts webpage here and use Pandas to scrape the table containing facts about the planet including Diameter, Mass, etc.Use Pandas to convert the data to a HTML table string.

In [10]:
url = 'https://space-facts.com/mars/'
# Read table data from url
facts_table = pd.read_html(url)

# Convert to dataframe
mars_facts_df = facts_table[0]
mars_facts_df.columns = ['Type', 'Measurement']
mars_facts_df

,Type,Measurement
0,Equatorial Diameter:,"6,792 km"
1,Polar Diameter:,"6,752 km"
2,Mass:,6.39 × 10^23 kg (0.11 Earths)
3,Moons:,2 (Phobos & Deimos)
4,Orbit Distance:,"227,943,824 km (1.38 AU)"
5,Orbit Period:,687 days (1.9 years)
6,Surface Temperature:,-87 to -5 °C
7,First Record:,2nd millennium BC
8,Recorded By:,Egyptian astronomers
